计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:0')

(device(type='cpu'),
 <torch.cuda.device at 0x1fcffbc53d0>)

查询可用gpu的数量

In [3]:
torch.cuda.device_count()

1

这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [4]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())
    ]


try_gpu(), try_gpu(0), try_all_gpus()

(device(type='cuda', index=0), device(type='cuda', index=0), None)

In [5]:
torch.cuda.is_available()

True

查询张量所在的设备

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

存储在GPU上

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

第二个GPU上创建一个随机张量

In [11]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.1997, 0.9591, 0.8330],
        [0.1025, 0.5751, 0.9834]], device='cuda:0')

要计算X + Y，我们需要决定在哪里执行这个操作

In [12]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


现在数据在同一个GPU上 (和都在)，我们可以将它们相加

In [13]:
Y + Z

tensor([[1.1997, 1.9591, 1.8330],
        [1.1025, 1.5751, 1.9834]], device='cuda:0')

In [14]:
Z.cuda(0) is Z

True

神经网络与GPU

In [15]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[0.7850],
        [0.7850]], device='cuda:0', grad_fn=<AddmmBackward0>)

确认模型参数存储在同一个GPU上

In [16]:
net[0].weight.data.device

device(type='cuda', index=0)